In [2]:
import pandas as pd
import re
from feature.base_dataset import limit_range
from glob import glob

In [6]:
def moving_average(input_path):
    """make moving_average dataset

    Args:
        input_path (str): data input path

    Returns:
        DataFrame: moving_average time dataset
    """
    input_list = sorted(glob(input_path))
    df_moving = pd.DataFrame()
    for i in input_list:
        df = pd.read_csv(i)
        df = limit_range(df)
        df = df.fillna(method='ffill')
        df = df.drop(['DAT', 'obs_time', '시간당분무량', '시간당백색광량',
                    '시간당적색광량', '시간당청색광량', '시간당총광량',
                    '일간누적청색광량', '일간누적적색광량', '일간누적백색광량'], axis=1)
           
        ma = df.rolling(6, min_periods=1).mean()
        ma['time'] = [i%24 for i in range(len(ma))]
        ma['DAT'] = [i//24 for i in range(len(ma))]

        df = pd.pivot_table(ma, index=['DAT'], columns=['time'], aggfunc='mean')
        df.columns = [''.join(str(col)) for col in df.columns]
        df = df.reset_index()    
        
        df_moving = pd.concat([df_moving, df])
    df_moving = df_moving.drop(['DAT'], axis=1)
    return df_moving
    

In [12]:
from glob import glob
train_input_list = sorted(glob('./data/train_input/*.csv'))
train_target_list = sorted(glob('./data/train_target/*.csv'))

test_input_list = sorted(glob('./data/test_input/*.csv'))
test_target_list = sorted(glob('./data/test_target/*.csv'))

In [7]:
train = moving_average('./data/train_input/*.csv')
# test = moving_average(test_input_list)

In [14]:
train.to_csv('train_roling.csv', index=False)
test.to_csv('test_roling.csv', index=False)